In [1]:
import os

In [2]:
%pwd

'd:\\Text summarization project\\Text-Summarizer-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Text summarization project\\Text-Summarizer-'

In [5]:
# !pip install ipykernel

In [6]:
## Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [7]:
## Configuration manager. 

from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir, 
            data_path = config.data_path, 
            model_ckpt=config.model_ckpt, 
            num_train_epochs = params.num_train_epochs, 
            warmup_steps = params.warmup_steps, 
            per_device_train_batch_size=params.per_device_train_batch_size, 
            weight_decay = params.weight_decay, 
            logging_steps = params.logging_steps, 
            evaluation_strategy = params.evaluation_strategy, 
            eval_steps = params.eval_steps, 
            save_steps = params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )  
        
        return model_trainer_config 

In [8]:
# pip install torch 


In [9]:
## Components
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, BartTokenizer
from datasets import load_dataset, load_from_disk
import torch

c:\Users\ASUS\Anaconda3\envs\texts\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-06 11:08:22,718: INFO: config: PyTorch version 2.6.0+cu118 available.]


In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
        
    def train(self):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        tokenizer = BartTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        model_pegasus.gradient_checkpointing_enable()

        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        ## Loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        print(dataset_samsum_pt)
        
        trainer_args = TrainingArguments(
            output_dir = 'distilbart-samsum', num_train_epochs = 1, warmup_steps = 500,
            per_device_train_batch_size = 1, per_device_eval_batch_size = 1,
            weight_decay = 0.01, logging_steps = 10,
            evaluation_strategy = 'steps', eval_steps = 500, save_steps = 1e6,
            gradient_accumulation_steps = 16,
            fp16=True
            ) 
        
        trainer = Trainer(model=model_pegasus, args = trainer_args, 
                          tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt['test'],
                          eval_dataset=dataset_samsum_pt['validation'])
        
        trainer.train()
        
        # Save model
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))   
        
        

In [11]:
# pip install "transformers[torch]"


In [12]:
# !pip install --upgrade accelerate


In [13]:
## Pipeline

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e    

[2025-04-06 11:08:23,811: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-04-06 11:08:23,814: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-04-06 11:08:23,817: INFO: common: created directory at: artifacts]
[2025-04-06 11:08:23,823: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\ASUS\Anaconda3\envs\texts\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--sshleifer--distilbart-cnn-12-6. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})


c:\Users\ASUS\Anaconda3\envs\texts\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ASUS\AppData\Local\Temp\ipykernel_46492\2034398030.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args = trainer_args,


Step,Training Loss,Validation Loss


c:\Users\ASUS\Anaconda3\envs\texts\Lib\site-packages\transformers\modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
